# Plot ski resorts and SNOTEL stations

This is an exploratory notebook which creates a map visual of the SNOTEL stations and ski areas so they can be viewed in relationship to each other. It helps me to visually understand how SNOTEL station data may or may not be representative of snow depth for the various ski resorts.

Ski areas GeoJSON is from [openskimap.org](https://openskimap.org) and SNOTEL station data is from [AWDB REST API](https://wcc.sc.egov.usda.gov/awdbRestApi/swagger-ui/index.html).

In [1]:
import json
import folium

In [2]:
with open('ski_areas.geojson', 'r') as f:
    ski_areas = json.load(f)

In [3]:
wa_ski_sites = [area for area in ski_areas['features'] if area['properties']['location'] and area['properties']['location']['iso3166_2'] == 'US-WA']
len(wa_ski_sites)

66

In [4]:
wa_ski_areas = [area for area in wa_ski_sites if area['geometry']['type'] != 'Point']
len(wa_ski_areas)

17

In [5]:
with open("snotel-wa-stations.json", "r") as f:
    wa_snotel_stations = json.load(f)

In [6]:
min_lat, min_lon, max_lat, max_lon = [45, -124, 49, -116]

m = folium.Map(location=[(max_lat - min_lat)/2 + min_lat, (max_lon - min_lon)/2 + min_lon], zoom_start=7)

wa_ski_geojson = {
    'type': 'FeatureCollection',
    'features': wa_ski_areas
}

folium.GeoJson(
    data=wa_ski_geojson,
    popup=folium.GeoJsonPopup(fields=['name'])
).add_to(m)

for snotel_station in wa_snotel_stations:
    
    folium.CircleMarker(
        location=[snotel_station['latitude'], snotel_station['longitude']],
        popup=f"{snotel_station['name']} ({snotel_station['stationTriplet']})",
        radius=3,
        color='green',
        fill=True,
        fillOpacity=0.7
    ).add_to(m)

m